# Optional: Scikit-learn primer.
In this additional assignment, you will learn to use the scikit-learn library. It is highly recommended to go through this notebook before starting with the final assignment.

## Introduction
All algorithms, both learning and pre-processing, in scikit-learn have been implemented with the same `fit`, `predict` and `transform` API. As soon as you have learned this API you can use any algorithm without having to implement it on your own. For a given learning problem, you can then apply all those algortihms in the same way. The API also hides all the complex optimization choices that have to be made. You can control these by changing the hyper-parameters. The effects of these choices have been well documented in the API documentation and the provided tutorials of scikit-learn.  



## Dataset

In this assignment, we will use the Iris dataset to keep things simple.

In [1]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

## Using classifiers
Using a classifier in scikit-learn consist of 3 steps:
1. Initialize the model. During this step, you can already give it some default hyper-parameters.
2. Fitting the model on the training data.
3. Making predictions and/or evaluating the model.

### Create
Creating models is very easy in scikit-learn. All you have to do is create a new instance of the model's class.

$ \ex{1} $ Extent the list of models with the`SVC` and `LogisticRegression` algorithms. Give the SVM a `poly` kernel. Also, give both algorithms a regularization constant `C=0.5` and `random_state=42`.

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

random_state = 42

models = {
    "GaussianNB": GaussianNB(),
    "DummyClassifier": DummyClassifier(strategy="most_frequent"),
    "DecisionTreeClassifier": DecisionTreeClassifier(max_depth=None, min_samples_leaf=2, random_state=random_state),
    "KNeighborsClassifier": KNeighborsClassifier(n_neighbors=3, weights="distance"),
    # START ANSWER
    "SVM": SVC(C=0.5, kernel="poly", random_state=random_state),
    "LogisticRegression": LogisticRegression(C=0.5, random_state=random_state),
    # END ANSWER
}

assert "GaussianNB" in models and isinstance(models["GaussianNB"], GaussianNB), "There is no GaussianNB in models"
assert "DecisionTreeClassifier" in models and isinstance(models["DecisionTreeClassifier"], DecisionTreeClassifier), "There is no DecisionTreeClassifier in models"
assert "KNeighborsClassifier" in models and isinstance(models["KNeighborsClassifier"], KNeighborsClassifier), "There is no KNeighborsClassifier in models"
assert "SVM" in models and isinstance(models["SVM"], SVC), "There is no SVC in models"
assert "LogisticRegression" in models and isinstance(models["LogisticRegression"], LogisticRegression), "There is no LogisticRegression in models"

### Fit
$ \ex{2} $ Fit each of your models on the entire training set by calling the `.fit` method of the model.

In [3]:
for name, model in models.items():
    # START ANSWER
    models[name] = model.fit(X, y)
    # END ANSWER

In [4]:
from sklearn.utils.validation import check_is_fitted

for model in models.values():
    check_is_fitted(model)

### Evaluate
The `sklearn.metrics` module has lots of metrics that can evaluate a model's predictions. Here is an example of how to calculate a model's F1 and accuracy score.

In [5]:
from sklearn.metrics import f1_score, accuracy_score

for name, model in models.items():
    prediction = model.predict(X)
    f1_score_value = f1_score(prediction, y, average="weighted")
    accuracy = accuracy_score(prediction, y)
    print(name)
    print("- accuracy_score", accuracy)
    print("- f1_score", f1_score_value)

GaussianNB
- accuracy_score 0.96
- f1_score 0.96
DummyClassifier
- accuracy_score 0.3333333333333333
- f1_score 0.5
DecisionTreeClassifier
- accuracy_score 0.98
- f1_score 0.9800020002000202
KNeighborsClassifier
- accuracy_score 1.0
- f1_score 1.0
SVM
- accuracy_score 0.9733333333333334
- f1_score 0.973344004268374
LogisticRegression
- accuracy_score 0.9666666666666667
- f1_score 0.9666700003333667


## Data splitting
Models usually achieve a high evaluation score on the training set. However, this doesn't say anything about how well it generalizes to unseen data. So we usually evaluate models using either a test/validation split or k-fold validation. Scikit-learn also makes our life easier here by implementing both functions for us.

### Test/validation split
We can split datasets into training and test sets using the `train_test_split` function. The `test_size` parameter indicate the percentage of data that should go to the test set. The `stratify`  parameter indicate that the split should take the distribution of target labels `y` into account during the split. This parameter ensures that both the train and test have the same distribution of target variables.

$ \ex{3} $ The data has already been split into a training and a test set. Fit the model using the training set and evaluate them using the test set.

The result on the test set should roughly be equal to:

|                  Model |    F1 | Accuracy |
|-----------------------:|------:|---------:|
|             GaussianNB |  0.86 |     0.86 |
| DummyClassifier        | 0.33  | 0.5      |
| DecisionTreeClassifier | 0.866 |    0.866 |
| KNeighborsClassifier   | 1     | 1        |
| SVM                    | 0.93     | 0.934        |
| LogisticRegression     | 0.933 | 0.934    |

In [6]:
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True, stratify=y)

# START ANSWER 
for name, model in models.items():
    models[name] = model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_predict)
    F1 = f1_score(y_test, y_predict, average="weighted")
    print(name,' ',F1,' ',accuracy)
# END ANSWER 

GaussianNB   0.8666666666666667   0.8666666666666667
DummyClassifier   0.16666666666666666   0.3333333333333333
DecisionTreeClassifier   0.8666666666666667   0.8666666666666667
KNeighborsClassifier   1.0   1.0
SVM   0.9326599326599326   0.9333333333333333
LogisticRegression   0.9326599326599326   0.9333333333333333


## K-fold validation
Setting up k-fold validation is a bit more work but we can do it as follows:

In [7]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

def k_fold_fit_and_evaluate(X, y, model, scoring_method, n_splits=5):
    # define evaluation procedure
    cv = KFold(n_splits=n_splits, random_state=42, shuffle=True)
    # evaluate model
    scores = cross_validate(model, X, y, scoring=scoring_method, cv=cv, n_jobs=-1)
    
       
    return scores["test_score"]

Note: `cross_validate` expects a `scoring_method`. We can create a `scoring_method` using the `make_scorer` function from scikit-learn.

$ \ex{4} $ Use the example below to calculate the mean and std for both the F1 and the accuracy score. The `k_fold_fit_and_evaluate` method returns the resulting k-fold validation score from the provided `scoring_method`.

Hint: use `np.mean` and `np.std`.

The result using k-fold validation should roughly be equal to:


|                  Model | mean F1 | std F1 | mean Accuracy | std Accuracy |
|-----------------------:|--------:|--------|--------------:|--------------|
|             GaussianNB |   0.959 | 0.0249 |        0.960 | 0.0249        |
| DummyClassifier        | 0.107   | 0.0187 | 0.26        | 0.0249       |
| DecisionTreeClassifier |   0.946 | 0.0338 |       0.94655 | 0.0338       |
| KNeighborsClassifier   | 0.966   | 0.0214 | 0.9663        | 0.02144      |
| SVM                    | 0.980   | 0.0163 | 0.980        | 0.0163      |
| LogisticRegression     | 0.966   | 0.0298 | 0.966        | 0.0298     |

In [8]:
from sklearn.metrics import make_scorer
import numpy as np

n_splits = 5


scoring_method_F1 = make_scorer(lambda prediction, true_target: f1_score(prediction, true_target, average="weighted"))
# START ANSWER 
scoring_method_acc = make_scorer(lambda prediction, true_target: accuracy_score(prediction, true_target))
# END ANSWER


for name, model in models.items():
    print(name)
    metrics_F1 = k_fold_fit_and_evaluate(X, y, model, scoring_method_F1, n_splits=n_splits)
    # START ANSWER
    metrics_acc = k_fold_fit_and_evaluate(X, y, model, scoring_method_acc, n_splits=n_splits)
    print(" mean F1:", '\t', " std F1")
    print(format(np.mean(metrics_F1),"5f"), '\t', format(np.std(metrics_F1),"5f"))
    print(" mean Accuracy", '\t', " std Accuracy")
    print(format(np.mean(metrics_acc),"5f"), '\t', format(np.std(metrics_acc),"5f"))
    print()
    # END ANSWER 

GaussianNB
 mean F1: 	  std F1
0.959916 	 0.024923
 mean Accuracy 	  std Accuracy
0.960000 	 0.024944

DummyClassifier
 mean F1: 	  std F1
0.107920 	 0.018663
 mean Accuracy 	  std Accuracy
0.260000 	 0.024944

DecisionTreeClassifier
 mean F1: 	  std F1
0.946560 	 0.033900
 mean Accuracy 	  std Accuracy
0.946667 	 0.033993

KNeighborsClassifier
 mean F1: 	  std F1
0.966362 	 0.021444
 mean Accuracy 	  std Accuracy
0.966667 	 0.021082

SVM
 mean F1: 	  std F1
0.979979 	 0.016348
 mean Accuracy 	  std Accuracy
0.980000 	 0.016330

LogisticRegression
 mean F1: 	  std F1
0.966667 	 0.029814
 mean Accuracy 	  std Accuracy
0.966667 	 0.029814



## Grid search
Scikit-learn also makes it easier to tune <span class="mark">hyper-parameters</span> using `GridSearchCV`.

$ \ex{5} $ Extend the `model_parameters` dict by specifying a grid search for the `KNeighborsClassifier`, <span class="mark">`SVM`</span> and the `LogisticRegression` models.

In [9]:
from sklearn.model_selection import GridSearchCV

random_state = 42
n_splits = 5
scoring_method = make_scorer(lambda prediction, true_target: f1_score(prediction, true_target, average="weighted"))

model_parameters = {
    "GaussianNB": {
    
    },
    "DummyClassifier": {
        
    },
    "DecisionTreeClassifier": {
        'random_state': [random_state],
        'max_depth': [None, 2, 5, 10]
    },
    # START ANSWER
    "KNeighborsClassifier": {
        "n_neighbors": [i for i in range(1,11)],
        "weights": ["uniform", "distance"] 
    },
    "SVM": {
        "C": [i/10 for i in range(1,11)],
#         "kernal": ["linear", "poly", "rbf", "sigmoid", "precomputed"],
        "random_state": [random_state]
    },
    "LogisticRegression": {
        "C": [i/10 for i in range(1,11)],
        "random_state": [random_state]
    }
    # END ANSWER
}

for model_name, parameters in model_parameters.items():
    model = models[model_name]
    
    cv = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    grid_search = GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=False, scoring=scoring_method).fit(X, y)
    
    best_model = grid_search.best_estimator_
    best_score = grid_search.best_score_
    best_params = grid_search.best_params_
    
    print(model_name)
    print("- best_score =", best_score)
    print("best paramters:")
    for k,v in best_params.items():
        print("-", k, v)


GaussianNB
- best_score = 0.9599161225248183
best paramters:
DummyClassifier
- best_score = 0.10791990370937739
best paramters:
DecisionTreeClassifier
- best_score = 0.9465598893859765
best paramters:
- max_depth None
- random_state 42
KNeighborsClassifier
- best_score = 0.9797720797720798
best paramters:
- n_neighbors 7
- weights distance
SVM
- best_score = 0.9799785345717235
best paramters:
- C 0.2
- random_state 42
LogisticRegression
- best_score = 0.9732912280701754
best paramters:
- C 0.7
- random_state 42


D:\Programmer\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Using Transformers
The transformers have a similar but slightly different API than the models. Transformers still have the `fit` method. The fit method is, for example, use in the `StandardScaler` to find the `mean` and `std` values. However, the `predict` method is replaced with the `transform` method. Scikit-learn did this to make it clear to the users that this is not a model but a feature transformer.


<span class="mark">Q: Why transform? Which method needs transformation? The transforming objects?</span>

In [10]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
# 概率模型（树形模型）不需要归一化,
# 因为它们不关心变量的值，而是关心变量的分布和变量之间的条件概率，
# 如决策树、RF。而像Adaboost、SVM、LR、Knn、KMeans之类的最优化问题就需要归一化。
scaler.mean_, scaler.scale_

(array([5.82962963, 3.05703704, 3.75111111, 1.20518519]),
 array([0.82210877, 0.44297659, 1.74999965, 0.763842  ]))

After fitting the transformer, you can call the `transform` method, and it will transform the input features based on the parameters it found during the last `fit` call.

In [11]:
X_train_transformed = scaler.transform(X_train)
print("X_train")
print("mean", X_train.mean())
print("std", X_train.std())
print()
print("X_train_transformed")
print("mean", X_train_transformed.mean())
print("std", X_train_transformed.std())

X_train
mean 3.460740740740741
std 1.9662465199534571

X_train_transformed
mean 6.579099405186112e-17
std 1.0


$ \ex{6} $ First, transform the dataset using the `Normalizer` transformer. The fit and evaluate each model using the transformed features.

In [12]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True, stratify=y)

scaler = preprocessing.Normalizer().fit(X_train)

# START ANSWER
X_train_Normalized = scaler.transform(X_train)
print("X_train")
print("mean", X_train.mean())
print("std", X_train.std())
print()
print("X_train_Normalized")
print("mean", X_train_Normalized.mean())
print("std", X_train_Normalized.std())
print()

# for name, model in models.items():
#     model_original = model.fit(X_train, y_train)
#     y_predict = model_original.predict(X_test)
#     F1 = f1_score(y_test, y_predict, average="weighted")
#     print(name,' ',F1)
    
#     model_nomalized = model.fit(X_train_Normalized, y_train)
#     y_predict_nomalized = model_nomalized.predict(X_test)
#     F1 = f1_score(y_test, y_predict_nomalized, average="weighted")
#     print("Nomalized " + str(name),' ',F1)
#     print()
# END ANSWER 

X_train
mean 3.460740740740741
std 1.9662465199534571

X_train_Normalized
mean 0.43844230430986214
std 0.24035046451267425



In [13]:
X, y = load_iris(return_X_y=True)

scaler_norm = preprocessing.Normalizer().fit(X,y)
scaler_stand = preprocessing.StandardScaler().fit(X,y)
X_norm = scaler_norm.transform(X)
X_stand = scaler_stand.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.1, 
                                                    random_state=42, 
                                                    shuffle=True, 
                                                    stratify=y
                                                   )
X_train_norm, X_test_norm, y_train, y_test = train_test_split(X_norm, y, 
                                                              test_size=0.1, 
                                                              random_state=42, 
                                                              shuffle=True, 
                                                              stratify=y
                                                             )
X_train_stand, X_test_stand, y_train, y_test = train_test_split(X_stand, y, 
                                                              test_size=0.1, 
                                                              random_state=42, 
                                                              shuffle=True, 
                                                              stratify=y
                                                             )

for name, model in models.items():
    model_original = model.fit(X_train, y_train)
    y_predict = model_original.predict(X_test)
    F1 = f1_score(y_test, y_predict, average="weighted")
    print(name,' ',F1)
    
    model_nomalized = model.fit(X_train_norm, y_train)
    y_predict_nomalized = model_nomalized.predict(X_test_norm)
    F1 = f1_score(y_test, y_predict_nomalized, average="weighted")
    print("Nomalized " + str(name),' ',F1)
    
    model_standerlized = model.fit(X_train_stand, y_train)
    y_predict_standerlized = model_nomalized.predict(X_test_stand)
    F1 = f1_score(y_test, y_predict_standerlized, average="weighted")
    print("Standardlized " + str(name),' ',F1)
    print()
    

GaussianNB   0.8666666666666667
Nomalized GaussianNB   1.0
Standardlized GaussianNB   0.8666666666666667

DummyClassifier   0.16666666666666666
Nomalized DummyClassifier   0.16666666666666666
Standardlized DummyClassifier   0.16666666666666666

DecisionTreeClassifier   0.8666666666666667
Nomalized DecisionTreeClassifier   0.7802197802197803
Standardlized DecisionTreeClassifier   0.8666666666666667

KNeighborsClassifier   1.0
Nomalized KNeighborsClassifier   1.0
Standardlized KNeighborsClassifier   0.9326599326599326

SVM   0.9326599326599326
Nomalized SVM   1.0
Standardlized SVM   0.861111111111111

LogisticRegression   0.9326599326599326
Nomalized LogisticRegression   0.861111111111111
Standardlized LogisticRegression   0.8666666666666667

